# OxMatch Algorithm 

### 1. Agents: 

#### First cluster (Stable Marriage Problem): Gale-Shapley
1. Proposing: Male looing for Female
2. Proposed: Female looing for Male

#### Second Cluster (Stable Roommate problem): 
3. Male looking for Male

#### Third Cluster  (Stable Roommate problem): :
4. Female looking for Female

#### Two matches for romance:
1. Similar
columns 15-33, except 24, 28. Least distance: larger sum of absolute difference, worse
2. Opposite

### 2. Algorithm

#### 2.1 Difference score: squared difference

Sum of squared difference between any two potential matches

#### 2.2 Deal breakers (column, question):

###### Change pair-wise preference score to +1000

    9. 'What years do you want to match with?'
    37. (38) "There are some political views I don't want to be matched with"
    50. I do not want to match with someone from my college
    51. I am happy to match with with someone from my course
    52. (53) "I don't want a match from..."

### 3. TODO
1. Parse qualitative answers (multiple choice format): Daniel Gagliardi
2. Add all deal breakers: Annie
3. Non-binary groups: Yuan Yuan Sun 
4. MM/FF: Giacomo Rossetti (Yuan Yuan Sun - Annie will give library link)
5. Statistics

In [435]:
import pandas as pd
import numpy as np
import itertools
from itertools import *

from matching.games import StableMarriage

In [740]:
MT2020 = pd.read_csv('anonymMT2020.csv',index_col=[0])

In [744]:
MT2020.iloc[:,8].value_counts()

Philosophy, Politics and Economics (PPE)     174
Law (Jurisprudence)                          172
History                                      149
English Language and Literature              148
Medicine                                     142
Chemistry                                    130
Engineering Science                          114
Modern Languages                             114
Physics                                      105
Mathematics                                   93
Biology                                       84
Biochemistry (Molecular and Cellular)         75
Geography                                     74
Economics and Management                      61
Classics                                      61
Music                                         41
History and Politics                          39
Materials Science                             32
Biomedical Sciences                           32
Modern Languages and Linguistics              32
Earth Sciences (Geol

In [739]:
#percentage of PPE students represented in OxMatch
174/737

0.23609226594301222

In [714]:
#gender
MT2020.iloc[:,2].value_counts()

Female                 1286
Male                   1025
Prefer not to say         7
Nonbinary                 3
Non-binary                2
Non Binary                2
Nonbinary (she/her)       1
Non-binary                1
nonbinary                 1
Non binary                1
attack helicopter         1
Nonbinary                 1
queer                     1
I don't                   1
Non                       1
Name: I identify as..., dtype: int64

In [715]:
#what gender are people looking for: 
MT2020.iloc[:,4].value_counts()

Male                       1146
Female                      881
Male, Female, Nonbinary     117
Male, Female                 96
Female, Nonbinary            30
Female, Male                 21
Male, Nonbinary              12
Female, Male, Nonbinary      10
Nonbinary                     6
Female, Nonbinary, Male       4
Nonbinary, Female, Male       4
Male, Nonbinary, Female       3
Nonbinary, Female             3
Nonbinary, Male, Female       1
Name: I want to match with people who are, dtype: int64

In [745]:
list(MT2020.columns)

['name',
 'What college are you at?',
 'I identify as...',
 'I want to be matched with people looking for...',
 'I want to match with people who are',
 'How do you feel about an open/polyamorous relationship?',
 'What year are you in?',
 'What years do you want to match with?',
 'What do you study?',
 'Where do you go?',
 'How COVID-Secure is your Date?',
 'What do you talk about?',
 'How far do you go?',
 "It's ok to leave friends on read",
 'Capitalism has failed the planet',
 'University COVID guidelines are too harsh',
 'Sharking is predatory',
 'Drug laws should be loosened',
 'I am scared that if I express my opinions I will be cancelled',
 'Pissing in the sink is normal',
 'How smart are you compared to the average Oxford student?',
 'Was clubbing a large part of your social life?',
 'The Pre-Covid best night out was...',
 'How do you feel about alcohol?',
 'How do you feel about cigarettes?',
 'Is music important in your relationships?',
 'Who are your favourite musicians?',
 '

# 1. Dividing Subsets

In [675]:
#male looking for female
proposing = MT2020[(MT2020.iloc[:,2] == 'Male') & (MT2020.iloc[:,4] == 'Female')]
proposed = MT2020[(MT2020.iloc[:,2] == 'Female') & (MT2020.iloc[:,4] == 'Male')]

In [676]:
MT2020.iloc[:,2].unique()

array(['attack helicopter', 'Male', 'Female', 'Prefer not to say',
       'Nonbinary', 'Non', "I don't", 'Nonbinary ', 'Non-binary',
       'Non Binary', 'nonbinary ', 'queer', 'Nonbinary (she/her)',
       'Non-binary ', 'Non binary '], dtype=object)

In [441]:
#157 people
MM = MT2020[(MT2020.iloc[:,4] == 'Male') & (MT2020.iloc[:,6] == 'Male')]

#72 people
FF = MT2020[(MT2020.iloc[:,4] == 'Female') & (MT2020.iloc[:,6] == 'Female')]

# 2. F looking for F, M looking for M

72 people, randomly divided into 2 groups

In [ ]:
#TODO

# 3. M looking for F & F looking for M 


In [703]:
def preference_table(df1,df2):
    """
    Take into two dataframes (proposing and proposed), return DataFrame of cartesian product of len(df1) * len(df2)  
    
    """
    
    #helper function: column used to match 
    def prase_df(df):
        parsedDf = df.iloc[:,13:31].drop(columns=df.columns[[22,26]]).fillna(0)
        return parsedDf
     
    parsedDf1 = prase_df(df1)
    parsedDf2 = prase_df(df2)
    
    matrix = np.zeros((len(df1),len(df2)))
    for i in range(len(df1)):
        for j in range(len(df2)):
            
            #Dealbreaker: Year 
            if (df1.iloc[i,6] not in df2.iloc[j,7]) | (df2.iloc[j,6] not in df1.iloc[i,7]): 
                matrix[i,j] += 1e4
            
            #Dealbreaker: Political views 
#             if (df1.iloc[i,8] in df2.iloc[j,9]) | (df2.iloc[j,8] in df1.iloc[i,9]): 
            
            else: 
                matrix[i,j] = np.sum((parsedDf1.iloc[i] - parsedDf2.iloc[j])**2) 
    
    table = pd.DataFrame(matrix,index = parsedDf1.index,columns = parsedDf2.index)
    return table

def create_prefs(prefrence_table):
    men_prefs = {}
    women_prefs = {}
    for i in range(prefrence_table.shape[0]):
        men_prefs['Man '+str(prefrence_table.index[i])] = ['Woman ' + str(x) for x in prefrence_table.iloc[i].sort_values().index]
    for i in range(prefrence_table.shape[1]):
        women_prefs['Woman '+str(prefrence_table.columns[i])] = ['Man ' + str(x) for x in prefrence_table.iloc[:,i].sort_values().index]
    
    return men_prefs, women_prefs

In [747]:
MT2020[MT2020.iloc[:,35] == 'Very conservative']

,name,What college are you at?,I identify as...,I want to be matched with people looking for...,I want to match with people who are,How do you feel about an open/polyamorous relationship?,What year are you in?,What years do you want to match with?,What do you study?,Where do you go?,...,Is there a particular college you don't want a match from?,I don't want a match from...,How was the quiz?,Any feedback for next time?,Do you want to share any contact details with your match (other than your Oxford email)?,Submitted At,Token,Unnamed: 60,Unnamed: 61,Unnamed: 62
84,person 2247,Trinity College,Male,NaN,Female,Not interested,Third,"Second, Third, Fourth + / Post Grad",Modern Languages and Linguistics,Walk in the park,...,False,NaN,5,The crush match is a good idea,False,2020-10-17 19:40:49,3mz3dmtnkoqk9jxpc1k663mz3g9zvp7r,NaN,NaN,NaN
177,person 1887,Keble College,Male,NaN,Female,Not interested,Second,"Second, Third, First",Economics and Management,Coffee,...,False,NaN,5,No,False,2020-10-17 21:16:22,t4yr4762iapix5tt4yr47pw7su6i8y06,NaN,NaN,NaN
196,person 1137,St Hugh's College,Male,NaN,Female,Not interested,Third,"Third, Second, First",Biology,Drinks,...,False,NaN,5,NaN,True,2020-10-17 21:44:33,8c352pnmx5rve8c35n8uwgnh48gwdqq4,NaN,NaN,NaN
259,person 2334,The Queen's College,Male,NaN,Female,Not interested,Third,"Second, Third","Philosophy, Politics and Economics (PPE)",Walk in the park,...,True,St Hugh's College,4,Damn you people better have some good data sec...,True,2020-10-18 03:04:29,hvv89c3dqz8m1ds7dphvv89c3mi8a46z,NaN,NaN,NaN
288,person 148,Worcester College,Female,NaN,Male,Happy either-way,Fourth + / Post Grad,"Third, Fourth + / Post Grad",Chemistry,Drinks,...,False,NaN,4,Nope,False,2020-10-18 09:04:10,aligqu9bybn3ot2aligqut6xbabp31t5,NaN,NaN,NaN
315,person 1645,New College,Male,NaN,Male,Happy either-way,First,"First, Second, Third, Fourth + / Post Grad",Physics,Dinner,...,False,NaN,3,No very good,True,2020-10-18 10:43:17,e6n8x60komkxthoyy2e6n8x60mbwuwn6,NaN,NaN,NaN
378,person 2948,New College,Female,NaN,Male,Not interested,Second,"First, Second, Third",Medicine,Dinner,...,False,NaN,4,no,True,2020-10-18 14:12:43,kev8mokexm546frbtv3kev8mo7fhu5j4,NaN,NaN,NaN
447,person 2264,Oriel College,Male,NaN,Female,Must have,Second,"Second, First, Third, Fourth + / Post Grad",Physics,Walk in the park,...,False,NaN,5,Not Particularly,True,2020-10-18 15:38:16,pi67p18ksh5771rtpi67p188fuq6n9xq,NaN,NaN,NaN
583,person 2493,St Edmund Hall,Prefer not to say,Nonbinary,Nonbinary,Must have,First,Fourth + / Post Grad,Biomedical Sciences,Walk in the park,...,False,NaN,5,No,True,2020-10-18 20:15:09,c5qmdcs74f5lkviuou2c5qmnwvvjzfyr,NaN,NaN,NaN
597,person 2796,St Antony's College,Male,NaN,Female,Not interested,Fourth + / Post Grad,"First, Second, Third, Fourth + / Post Grad","Philosophy, Politics and Economics (PPE)",Coffee,...,False,NaN,4,No,True,2020-10-18 20:31:12,m0dcx9veubdm887n5zm0dcxal5kvatnh,NaN,NaN,NaN


In [707]:
proposing[:10]

,name,What college are you at?,I identify as...,I want to be matched with people looking for...,I want to match with people who are,How do you feel about an open/polyamorous relationship?,What year are you in?,What years do you want to match with?,What do you study?,Where do you go?,...,Is there a particular college you don't want a match from?,I don't want a match from...,How was the quiz?,Any feedback for next time?,Do you want to share any contact details with your match (other than your Oxford email)?,Submitted At,Token,Unnamed: 60,Unnamed: 61,Unnamed: 62
1,person 60,Lady Margaret Hall,Male,NaN,Female,Not interested,Third,"Second, Third, Fourth + / Post Grad",Law (Jurisprudence),Drinks,...,False,NaN,4,Nah,True,2020-10-17 17:59:29,uubh5nxvl9zjpupaosueuubhs2y15tyo,NaN,NaN,NaN
4,person 280,St Hugh's College,Male,NaN,Female,Not interested,Second,"Second, Third, First",History (Ancient and Modern),Coffee,...,False,NaN,3,no,False,2020-10-17 19:05:19,p4e53wo7urugw2r17cp4e53wxt3k9aft,NaN,NaN,NaN
7,person 2789,Harris Manchester College,Male,NaN,Female,Not interested,Fourth + / Post Grad,"Third, Fourth + / Post Grad, Second",Oriental Studies,Drinks,...,False,NaN,5,No,True,2020-10-17 19:07:19,w78q1ej3wfkjylfzknv7vw78q1ejf3an,NaN,NaN,NaN
10,person 862,Worcester College,Male,NaN,Female,Not interested,First,"First, Second",Music,Dinner,...,False,NaN,4,Not particularly,True,2020-10-17 19:09:28,y9sga4hjo5v24pzopy9sgatubicdry1j,NaN,NaN,NaN
15,person 540,St Hugh's College,Male,NaN,Female,Not interested,Second,"First, Second, Third",Economics and Management,Drinks,...,False,NaN,3,Nah,False,2020-10-17 19:11:01,lixz33klcikt5a8lixz3qx2cdsh57l81,NaN,NaN,NaN
28,person 1676,St Hilda's College,Male,NaN,Female,Happy either-way,Third,"Second, Third, Fourth + / Post Grad, First",Physics,Walk in the park,...,False,NaN,5,Ahmed's has no competition. Please don't insul...,False,2020-10-17 19:15:11,xknocoosuwneezcyb5y9ol0vxknocoo1,NaN,NaN,NaN
29,person 276,Somerville College,Male,NaN,Female,Happy either-way,Fourth + / Post Grad,"Fourth + / Post Grad, Third, Second",Chemistry,Walk in the park,...,True,Christ Church,4,Nope,False,2020-10-17 19:16:24,jaai6eyskr7gbceye7lw5jaai6e5w37q,NaN,NaN,NaN
33,person 1967,Magdalen College,Male,NaN,Female,Not interested,Second,Second,"Psychology, Philosophy and Linguistics",Drinks,...,False,NaN,4,No,True,2020-10-17 19:18:44,58fp6sl9gw7wwsomkw58fp6eizg58b7r,NaN,NaN,NaN
34,person 355,Harris Manchester College,Male,NaN,Female,Must have,Fourth + / Post Grad,"First, Second, Third, Fourth + / Post Grad",Theology and Religion,Coffee,...,False,NaN,5,Thanks for including the non-monogamy question!,False,2020-10-17 19:19:05,rq7e7f6pe6x6uhvqb4xrq7e7f6zoscje,NaN,NaN,NaN
35,person 210,Christ Church,Male,NaN,Female,Not interested,Third,"Third, Fourth + / Post Grad, Second",Engineering Science,Drinks,...,False,NaN,4,Nope,True,2020-10-17 19:19:57,8o59yvvi0qm2bfuj8o59cdrimhrsydod,NaN,NaN,NaN


In [708]:
proposed[:10]

,name,What college are you at?,I identify as...,I want to be matched with people looking for...,I want to match with people who are,How do you feel about an open/polyamorous relationship?,What year are you in?,What years do you want to match with?,What do you study?,Where do you go?,...,Is there a particular college you don't want a match from?,I don't want a match from...,How was the quiz?,Any feedback for next time?,Do you want to share any contact details with your match (other than your Oxford email)?,Submitted At,Token,Unnamed: 60,Unnamed: 61,Unnamed: 62
6,person 989,Magdalen College,Female,NaN,Male,Not interested,Second,"Second, Third, Fourth + / Post Grad",Medicine,Coffee,...,False,NaN,4,Nah it’s good!,False,2020-10-17 19:07:13,jmgrghnkgzznfdgjmggg5i3y3wk5ax2m,NaN,NaN,NaN
9,person 2628,Oriel College,Female,NaN,Male,Not interested,Third,"Third, Fourth + / Post Grad, Second",Computer Science,Drinks,...,False,NaN,4,No,False,2020-10-17 19:08:51,j72y38mvgr344lj72742amciaojt5u0y,NaN,NaN,NaN
12,person 1306,Magdalen College,Female,NaN,Male,Not interested,Second,"Second, Third, Fourth + / Post Grad",History,Drinks,...,False,NaN,5,💕✨💕✨,True,2020-10-17 19:09:58,aj9bfuo3qdy2fj3daj9bf06b07n89eqq,NaN,NaN,NaN
13,person 2351,Exeter College,Female,NaN,Male,Not interested,Third,"Third, Second, Fourth + / Post Grad",Medicine,Drinks,...,False,NaN,5,No,False,2020-10-17 19:10:19,w7abhhahj6bigw7abh0evxq1etrfgl5z,NaN,NaN,NaN
14,person 1381,Somerville College,Female,NaN,Male,Not interested,Second,"Third, Fourth + / Post Grad, Second","Philosophy, Politics and Economics (PPE)",Drinks,...,False,NaN,4,Excellent stuff,True,2020-10-17 19:10:35,weurx7elr2hweurx7f6do3yqz6v97l2y,NaN,NaN,NaN
18,person 758,Wadham College,Female,NaN,Male,Not interested,Fourth + / Post Grad,"Fourth + / Post Grad, Third",Law (Jurisprudence),Drinks,...,False,NaN,5,Nope,True,2020-10-17 19:11:17,5m8gik8tda7mlft5m8gikw984c8jhvgz,NaN,NaN,NaN
20,person 1730,Lady Margaret Hall,Female,NaN,Male,Not interested,Second,"Second, First",Biochemistry (Molecular and Cellular),Walk in the park,...,False,NaN,5,N/a,True,2020-10-17 19:11:25,tlvcdv3r2ebqrfs8d4rtlvcdvkvu7j1g,NaN,NaN,NaN
21,person 2546,Exeter College,Female,NaN,Male,Not interested,Third,"Second, Third, Fourth + / Post Grad",Medicine,Drinks,...,False,NaN,5,No,False,2020-10-17 19:11:44,woiqdslzs3e060fq7itwoiqdsz41kcb4,NaN,NaN,NaN
22,person 780,Magdalen College,Female,NaN,Male,Not interested,Third,"Second, Third, Fourth + / Post Grad",Modern Languages,Drinks,...,False,NaN,3,nothing,True,2020-10-17 19:12:17,k2y0whsqjmtffth7dck2y0kbag0nrpxk,NaN,NaN,NaN
23,person 2571,Trinity College,Female,NaN,Male,Not interested,Second,"Second, Third",Medicine,Walk in the park,...,False,NaN,4,Ask more questions about sport,True,2020-10-17 19:12:37,7n5nq4syn96n3yc7n5nq45j0j9rce23i,NaN,NaN,NaN


In [709]:
preference_table(proposing[:10],proposed[:10])

,6,9,12,13,14,18,20,21,22,23
1,44.0,28.0,40.0,37.0,35.0,25.0,10000.0,27.0,42.0,33.0
4,53.0,19.0,93.0,60.0,42.0,10000.0,41.0,74.0,45.0,70.0
7,76.0,34.0,26.0,37.0,37.0,29.0,10000.0,35.0,48.0,10000.0
10,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,30.0,10000.0,10000.0,10000.0
15,40.0,16.0,62.0,43.0,23.0,10000.0,24.0,47.0,26.0,47.0
28,44.0,18.0,52.0,39.0,39.0,35.0,10000.0,37.0,40.0,45.0
29,45.0,47.0,49.0,82.0,50.0,42.0,10000.0,54.0,51.0,10000.0
33,24.0,10000.0,68.0,10000.0,29.0,10000.0,22.0,10000.0,10000.0,41.0
34,50.0,40.0,36.0,43.0,67.0,41.0,10000.0,29.0,60.0,10000.0
35,35.0,19.0,41.0,22.0,38.0,36.0,10000.0,20.0,23.0,36.0


In [731]:
create_prefs(preference_table(proposing[:10],proposed[:10]))[1]

{'Woman 6': ['Man 33',
  'Man 35',
  'Man 15',
  'Man 1',
  'Man 28',
  'Man 29',
  'Man 34',
  'Man 4',
  'Man 7',
  'Man 10'],
 'Woman 9': ['Man 15',
  'Man 28',
  'Man 4',
  'Man 35',
  'Man 1',
  'Man 7',
  'Man 34',
  'Man 29',
  'Man 10',
  'Man 33'],
 'Woman 12': ['Man 7',
  'Man 34',
  'Man 1',
  'Man 35',
  'Man 29',
  'Man 28',
  'Man 15',
  'Man 33',
  'Man 4',
  'Man 10'],
 'Woman 13': ['Man 35',
  'Man 1',
  'Man 7',
  'Man 28',
  'Man 15',
  'Man 34',
  'Man 4',
  'Man 29',
  'Man 10',
  'Man 33'],
 'Woman 14': ['Man 15',
  'Man 33',
  'Man 1',
  'Man 7',
  'Man 35',
  'Man 28',
  'Man 4',
  'Man 29',
  'Man 34',
  'Man 10'],
 'Woman 18': ['Man 1',
  'Man 7',
  'Man 28',
  'Man 35',
  'Man 34',
  'Man 29',
  'Man 4',
  'Man 10',
  'Man 15',
  'Man 33'],
 'Woman 20': ['Man 33',
  'Man 15',
  'Man 10',
  'Man 4',
  'Man 1',
  'Man 7',
  'Man 28',
  'Man 29',
  'Man 34',
  'Man 35'],
 'Woman 21': ['Man 35',
  'Man 1',
  'Man 34',
  'Man 7',
  'Man 28',
  'Man 15',
  'Man 29'

In [712]:
create_prefs(preference_table(proposing[:10],proposed[:10]))[0]

{'Man 1': ['Woman 18',
  'Woman 21',
  'Woman 9',
  'Woman 23',
  'Woman 14',
  'Woman 13',
  'Woman 12',
  'Woman 22',
  'Woman 6',
  'Woman 20'],
 'Man 4': ['Woman 9',
  'Woman 20',
  'Woman 14',
  'Woman 22',
  'Woman 6',
  'Woman 13',
  'Woman 23',
  'Woman 21',
  'Woman 12',
  'Woman 18'],
 'Man 7': ['Woman 12',
  'Woman 18',
  'Woman 9',
  'Woman 21',
  'Woman 13',
  'Woman 14',
  'Woman 22',
  'Woman 6',
  'Woman 20',
  'Woman 23'],
 'Man 10': ['Woman 20',
  'Woman 6',
  'Woman 9',
  'Woman 12',
  'Woman 13',
  'Woman 14',
  'Woman 18',
  'Woman 21',
  'Woman 22',
  'Woman 23'],
 'Man 15': ['Woman 9',
  'Woman 14',
  'Woman 20',
  'Woman 22',
  'Woman 6',
  'Woman 13',
  'Woman 21',
  'Woman 23',
  'Woman 12',
  'Woman 18'],
 'Man 28': ['Woman 9',
  'Woman 18',
  'Woman 21',
  'Woman 13',
  'Woman 14',
  'Woman 22',
  'Woman 6',
  'Woman 23',
  'Woman 12',
  'Woman 20'],
 'Man 29': ['Woman 18',
  'Woman 6',
  'Woman 9',
  'Woman 12',
  'Woman 14',
  'Woman 22',
  'Woman 21',
  '

In [710]:
#Creating matches
men_prefs = create_prefs(preference_table(proposing[:10],proposed[:10]))[0]
women_prefs = create_prefs(preference_table(proposing[:10],proposed[:10]))[1]

game = StableMarriage.create_from_dictionaries(men_prefs,women_prefs)
result = game.solve()
pd.DataFrame(result.items()).astype(str)

,0,1
0,Man 1,Woman 18
1,Man 4,Woman 14
2,Man 7,Woman 12
3,Man 10,Woman 23
4,Man 15,Woman 9
5,Man 28,Woman 13
6,Man 29,Woman 6
7,Man 33,Woman 20
8,Man 34,Woman 22
9,Man 35,Woman 21


In [358]:
preference_table = pd.DataFrame(a,index = parsedProposing.index,columns = parsedProposed.index)

#randomly drop the extra women
remove = np.random.choice(np.arange(988),size=181,replace=False)
dropped_preference_table = preference_table.drop(preference_table.columns[remove],axis=1)
dropped_preference_table

,6,9,12,13,14,18,20,21,22,23,...,2311,2312,2318,2319,2320,2323,2329,2330,2331,2332
1,18.0,18.0,20.0,15.0,17.0,15.0,24.0,13.0,18.0,19.0,...,21.0,15.0,12.0,20.0,23.0,20.0,18.0,21.0,18.0,19.0
4,23.0,13.0,33.0,28.0,20.0,26.0,19.0,30.0,21.0,28.0,...,28.0,24.0,29.0,13.0,18.0,17.0,21.0,20.0,23.0,36.0
7,26.0,18.0,16.0,21.0,19.0,17.0,26.0,19.0,20.0,21.0,...,21.0,17.0,22.0,24.0,25.0,22.0,18.0,25.0,20.0,21.0
10,20.0,14.0,24.0,23.0,19.0,23.0,16.0,21.0,20.0,21.0,...,17.0,19.0,26.0,12.0,13.0,12.0,16.0,21.0,14.0,23.0
15,20.0,12.0,26.0,19.0,13.0,17.0,16.0,21.0,16.0,23.0,...,25.0,15.0,20.0,18.0,21.0,18.0,14.0,25.0,16.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316,16.0,12.0,26.0,21.0,13.0,15.0,18.0,21.0,16.0,23.0,...,19.0,19.0,20.0,12.0,15.0,14.0,16.0,19.0,14.0,21.0
2324,31.0,21.0,35.0,22.0,30.0,28.0,21.0,28.0,27.0,24.0,...,32.0,26.0,31.0,15.0,20.0,23.0,27.0,26.0,27.0,36.0
2325,20.0,22.0,14.0,17.0,23.0,17.0,26.0,11.0,18.0,19.0,...,15.0,15.0,16.0,20.0,25.0,20.0,16.0,25.0,18.0,13.0
2327,21.0,15.0,23.0,16.0,16.0,16.0,13.0,14.0,17.0,18.0,...,22.0,10.0,17.0,19.0,22.0,21.0,11.0,24.0,13.0,24.0


In [386]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(10000)

5000


In [388]:
from matching.games import StableMarriage
game = StableMarriage.create_from_dictionaries(hospital_prefs, resident_prefs)
result = game.solve()